#Evaluation of directions discovered

## Stage 1. Preparation


*   Cloning Repo
*   Installing Dependencies
*   Downloading pretrained models



In [1]:
!git clone https://github.com/yjiang666/gan_interpretability.git repo
%cd repo/

!pip3 install -r requirements.txt
!pip install ninja

%tensorflow_version 1.x

fatal: destination path 'repo' already exists and is not an empty directory.
/content/repo
TensorFlow 1.x selected.


In [2]:
!python download.py
!python loading.py

downloading stylegan2
from https://www.dropbox.com/s/vtygvy0xiw4uvmd/pretrained_stylegan2_ffhq_256_id_24_lpips_5.tar
--2021-08-16 06:43:28--  https://www.dropbox.com/s/vtygvy0xiw4uvmd/pretrained_stylegan2_ffhq_256_id_24_lpips_5.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vtygvy0xiw4uvmd/pretrained_stylegan2_ffhq_256_id_24_lpips_5.tar [following]
--2021-08-16 06:43:29--  https://www.dropbox.com/s/raw/vtygvy0xiw4uvmd/pretrained_stylegan2_ffhq_256_id_24_lpips_5.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc896972c61e14c8be0868f99e86.dl.dropboxusercontent.com/cd/0/inline/BUUR39_LnYHRSWflo_n8NC917HwCmPeoSwwtj9RICQO0S3tnmGBAYNyZYD64OQ6VdN225ugLFB_jAS35ca0fZIGK0YobgBgZnJ0kjSn0xoUywreVSsOMksNy_pYPbfo8Ggv9L2Il5F

In [3]:
ptfiles = []
import os
for dirpath, subdirs, files in os.walk("/content/repo/"):
    for x in files:
        if x.endswith(".pt"):
            ptfiles.append(os.path.join(dirpath, x))
print(ptfiles)            

['/content/repo/models/pretrained/deformators/StyleGAN2/models/deformator_30000.pt', '/content/repo/models/pretrained/generators/StyleGAN2/ffhq-256-config-e-003810.pt']


In [4]:
%cd /content/repo/
%load_ext autoreload
%autoreload 2

import torch
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from torch_tools.visualization import to_image
from visualization import interpolate
from loading import load_from_dir
%matplotlib inline
import dnnlib.tflib as tflib

tflib.init_tf()

/content/repo


##Stage 2. Evaluation



*   Loading models
*   Generating GIFs for all directions



In [5]:
deformator, G, shift_predictor = load_from_dir(
    '/content/repo/models/pretrained/deformators/StyleGAN2/',
    G_weights='/content/repo/models/pretrained/generators/StyleGAN2/ffhq-256-config-e-003810.pt')


In [6]:
!pip install Pillow

In [7]:
from utils import is_conditional
import numpy as np
from PIL.Image import Image
import PIL.Image
from matplotlib.pyplot import imshow

dir_count = 30
id_loss = 22
lpip_loss = 3

RESULT_PATH = '../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_%s_id_%s_lpips_%s' % (dir_count, str(id_loss), str(lpip_loss))

In [8]:
def get_gif_for_dim(G, deformator, inspection_dir, RESULT_PATH):
    rows = 8
    gifs = []
    zs = torch.randn([rows, G.dim_z] if type(G.dim_z) == int else [rows] + G.dim_z, device='cuda')
    plt.figure(figsize=(5, rows), dpi=250)
    intpls = []
    for z, i in zip(zs, range(rows)):
        interpolation_deformed = interpolate(G, z.unsqueeze(0),
                                             shifts_r=16,
                                             shifts_count=3,
                                             dim=inspection_dir,
                                             deformator=deformator,
                                             with_central_border=True)
        frames = []
        intpls.append(interpolation_deformed)
        for item in interpolation_deformed:
            grid = make_grid(item, nrow=11, padding=1, pad_value=0)
            grid = torch.clamp(grid, -1, 1)
            image_grid = to_image(grid)
            frames.append(image_grid)
        gifs.append(frames)

    os.makedirs(RESULT_PATH, exist_ok=True)
    output_path = os.path.join(RESULT_PATH, "inspect_dir_%s.gif" % str(inspection_dir))

    frame_first = [[] for x in range(rows-1)]
    for index, gif in enumerate(gifs):
        for fnum, frame in enumerate(gif):
            frame_first[fnum].append(frame)
    frame_first_gifs = []
    for i, item in enumerate(frame_first):
        widths, heights = zip(*(i.size for i in item))
        total_width = sum(widths)
        max_height = max(heights)
        new_im = PIL.Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for im in item:
            new_im.paste(im, (x_offset, 0))
            x_offset += im.size[0]
        frame_first_gifs.append(new_im)

    frame_first_gifs[0].save(output_path, format='GIF', append_images=frame_first_gifs[1:], save_all=True,
                             duration=600, loop=0)
    print("Saving gif to %s" % output_path)

In [9]:
if is_conditional(G):
    G.set_classes(12)

for i in range(dir_count):
    get_gif_for_dim(G, deformator, i, RESULT_PATH)

Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_0.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_1.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_2.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_3.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_4.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_5.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_6.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_7.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_8.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_9.gif
Saving gif

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_20.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_21.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_22.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_23.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_24.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_25.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_26.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_27.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_28.gif
Saving gif to ../drive/MyDrive/GAN_Latent_Space_Output_Gifs/dir_30_id_22_lpips_3/inspect_dir_29.gif


<Figure size 1250x2000 with 0 Axes>

<Figure size 1250x2000 with 0 Axes>

...

<Figure size 1250x2000 with 0 Axes>

<Figure size 1250x2000 with 0 Axes>

<Figure size 1250x2000 with 0 Axes>

<Figure size 1250x2000 with 0 Axes>

<Figure size 1250x2000 with 0 Axes>